In [1]:
    from wtypes import *
    import pytest, typing
    invalid = pytest.raises(ValidationError)

In [2]:
    def test_partial_defaults():
        class d(Dict):
            i: Integer = 20

        assert d(j=9) == {'i': 20, 'j': 9}
        assert d(i=9) == {'i': 9}


In [3]:
    def test_nested_schema():
        class c(Dict): a: object
        class d(c): b: Integer
        assert c._schema.toDict() =={'properties': {'a': {}}, 'type': 'object'}
        d._schema.toDict() =={'properties': {'a': {}, 'b': {'type': 'integer'}}, 'type': 'object'}

In [4]:
    def test_update_attr():
        
        for cls in [Bunch]:
            class Class(cls): 
                a: int = 1
            d = Class({})
            with invalid: d['a'] = 'abc'
            assert d['a'] == 1
            d.a = 10
            assert d.a == 10

In [5]:
    def test_dict():
        class Thing(Dict):
            a: Integer

        Thing._schema.toDict() == {'properties': {'a': {'type': 'integer'}}, 'required': ['a'], 'type': 'object'}
        assert Thing(a=1)
        with invalid:
            Thing(a='abc')


In [6]:
    def test_bunch():
        class Thing(Bunch):
            a: Integer

        Thing._schema.toDict() == {'type': 'object', 'properties': {'a': {'type': 'integer'}}, 'required': ['a']}
        t = Thing(a=1)
        assert t.a == 1
        with invalid:
            Thing(a='abc')


In [7]:
    def test_dict_mods():
        d = Dict[{'a': Integer, 'b': String}]({'a': 1})

        with invalid: d["a"] = 'abc'
        d['a'] = 8
        d['b'] = 'abc'
        with invalid: d['b'] = 10
        d['b'] = 'wxyz'

        assert d == {'a': 8, 'b': 'wxyz'}


In [8]:
    def test_list_mods():
        l = List[Integer, Bool]()
        with invalid: l.append(1.2)
        l.append(1)
        l.extend([1,True])
        with invalid: l.extend([1, 'abc'])
        l.extend([False, 2, 4])
        assert l == [1, 1, True, False, 2, 4]
        with invalid: l.insert(1, 'abc')
        l.insert(1, 8)
        assert l == [1, 8, 1, True, False, 2, 4]

        ml = List[Integer] + MinItems[1]

        with invalid: ml()
        with invalid: ml(['abc'])
        assert ml([1])

In [9]:
    def test_update_item():
        for cls in [Dict, Bunch]:
            d = cls[{'a': int}]({})
            assert 'a' not in d
            d['a'] = 10
            assert d['a'] == 10

In [10]:
    def test_load_config():
        class BadProject(Bunch):
            tool: Dict
            __annotations__ = {'build-system': Integer}

        with invalid:
            BadProject().from_config_file('pyproject.toml')
        
        class PyProject(Bunch):
            tool: Dict
            __annotations__ = {'build-system': Dict}
        PyProject.from_config_file('pyproject.toml')

In [11]:
    def test_tuple():
        assert Tuple((1,2))
        assert Tuple([1,2])
        with invalid: Tuple[[Integer, String]]({})

In [12]:
    def test_complex_dict():
        with invalid: (Dict[{'a': Email}] + Required['a']+ Default[{}] + AdditionalProperties[False])({})
        with invalid: (Dict[{'a': Email}] + Default[{}] + AdditionalProperties[False])({'a': 'no'})
        assert (Dict[{'a': Email}] + Default[{}] + AdditionalProperties[False])({'a': 'no@yes'})

In [13]:
    typing.Union[Integer, Forward[range]].__args__

(wtypes.base.Integer, abc.Forward)

In [14]:
    def test_dataclass():
        class Thing(DataClass):
            a: typing.Union[]

        assert dataclasses.is_dataclass(Thing)
        Thing._schema.toDict() == {'type': 'object', 'properties': {'a': {'type': 'integer'}}, 'required': ['a']}
        assert Thing(a=1)
        with invalid:
            Thing(a='abc')


SyntaxError: invalid syntax (<ipython-input-14-1762cc1dce5d>, line 3)

In [ ]:
    def test_dict_additionalproperties():
        class d(Dict, additionalProperties=False):
            a: int
            b: float = 1.1
                
        with invalid: d({'c': 1})
        d({'a': 1})
        with invalid: d({'a': 1})['c'] = 1